<a href="https://colab.research.google.com/github/Hmd02/72_NotSlytherin_2/blob/main/72_NOTSLYTHERIN_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor
import tensorflow as tf

In [159]:
def createTrain():
  merged = pd.read_csv('/content/merged - merged.csv')
  merged.drop(['Unnamed: 0','ANNUAL','Jul-Oct', 'Nov-Mar', 'Apr-Jun', 'Total_Avg','Crop_Year'],axis=1,inplace=True)
  merged.drop(['District_Name'],axis=1,inplace=True)
  # print(merged.loc[merged["Crop"]=="Rice",:])
  # print(merged.loc[merged["Crop"]=="Wheat",:])
  # print(merged.loc[merged["Crop"]=="Jowar",:])
  # print(merged.loc[merged["Crop"]=="Bajra",:])
  # print(merged.loc[merged["Crop"]=="Coconut",:])
  # df=merged.copy()
  # df=merged.loc[(merged["Crop"]=="Rice")|(merged["Crop"]=="Wheat")|(merged["Crop"]=="Barley")|(merged["Crop"]=="Jowar"),:]
  # print(df.Production.count())
  train = merged.loc[:146000, :]
  test = merged.iloc[146000:, :]
  train.dropna(subset=['Production'],inplace=True,axis=0)
  # df=merged.loc[(merged["Crop"]=="Rice")|(merged["Crop"]=="Wheat")|(merged["Crop"]=="Barley")|(merged["Crop"]=="Jowar"),:]
  # print(df.count())
  test.drop('Production', axis=1,inplace=True)
  train.to_csv('train.csv')
  test.to_csv('test.csv')

In [160]:
createTrain()

In [161]:
def get_data():
    #get train data
    train_data_path ='/content/train.csv'
    train = pd.read_csv(train_data_path)
    
    #get test data
    test_data_path ='/content/test.csv'
    test = pd.read_csv(test_data_path)
    
    return train , test

def get_combined_data():
  #reading train data
  train , test = get_data()
  
  
  target = train.Production
  # print(train.isnull().sum())
  # print(target.isnull().sum())
  train.drop(['Production'],axis = 1 , inplace = True)
  # print(train.count())

  combined = train.append(test)
  combined.reset_index(inplace=True)
  combined.drop(combined.columns[combined.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
  combined.drop('index', axis=1, inplace = True)
  return combined, target

#Load train and test data into pandas DataFrames
train_data, test_data = get_data()

#Combine train and test data to process them together
combined, target = get_combined_data()

In [162]:
# def createTrain():
#   merged = pd.read_csv('/content/merged.csv')
#   train = merged.loc[:146000, :]
#   test = merged.iloc[146000:, :]
#   test.drop('Production', axis=1,inplace=True)
#   print(test['Total_Avg'])
#   train.to_csv('train.csv')
#   test.to_csv('test.csv')

In [163]:
# createTrain()

In [164]:
def split_combined():
    global combined
    train = combined[:144691]
    test = combined[144691:]

    return train , test

In [165]:
combined.columns

Index(['State_Name', 'Season', 'Crop', 'Area', 'Rainfall'], dtype='object')

In [166]:
def oneHotEncode(df,colNames):
    for col in colNames:
        if( df[col].dtype == np.dtype('object')):
            dummies = pd.get_dummies(df[col],prefix=col)
            df = pd.concat([df,dummies],axis=1)

            #drop the encoded column
            df.drop([col],axis = 1 , inplace=True)
    return df

In [167]:
print('There were {} columns before encoding categorical features'.format(combined.shape[1]))
combined = oneHotEncode(combined, combined.columns)
print('There are {} columns after encoding categorical features'.format(combined.shape[1]))

There were 5 columns before encoding categorical features
There are 153 columns after encoding categorical features


In [168]:
def get_cols_with_no_nans(df,col_type):
    '''
    Arguments :
    df : The dataframe to process
    col_type : 
          num : to only get numerical columns with no nans
          no_num : to only get nun-numerical columns with no nans
          all : to get any columns with no nans    
    '''
    if (col_type == 'num'):
        predictors = df.select_dtypes(exclude=['object'])
    elif (col_type == 'no_num'):
        predictors = df.select_dtypes(include=['object'])
    elif (col_type == 'all'):
        predictors = df
    else :
        print('Error : choose a type (num, no_num, all)')
        return 0
    cols_with_no_nans = []
    for col in predictors.columns:
        if not df[col].isnull().any():
            cols_with_no_nans.append(col)
    return cols_with_no_nans

In [169]:
num_cols = get_cols_with_no_nans(combined , 'num')
cat_cols = get_cols_with_no_nans(combined , 'no_num')

In [170]:
print ('Number of numerical columns with no nan values :',len(num_cols))
print ('Number of nun-numerical columns with no nan values :',len(cat_cols))

Number of numerical columns with no nan values : 153
Number of nun-numerical columns with no nan values : 0


In [171]:
# print('There were {} columns before encoding categorical features'.format(combined.shape[1]))
# combined = oneHotEncode(combined, combined.columns)
# print('There are {} columns after encoding categorical features'.format(combined.shape[1]))

In [172]:
#train.loc[:146000, 'Total_Avg']

In [173]:
train, test = split_combined()

In [174]:
NN_model = Sequential()

In [175]:
print(train.shape[1])

153


In [176]:
NN_model.add(Dense(153, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))

In [177]:
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

In [178]:
NN_model.add(Dense(1, kernel_initializer='normal',activation='softmax'))

In [179]:
# sgd = tf.keras.optimizers.SGD(learning_rate=0.011)

In [180]:
for col in combined.columns:
  if combined[col].isnull().sum()<0:
    print(col)

In [181]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 153)               23562     
                                                                 
 dense_21 (Dense)            (None, 256)               39424     
                                                                 
 dense_22 (Dense)            (None, 256)               65792     
                                                                 
 dense_23 (Dense)            (None, 256)               65792     
                                                                 
 dense_24 (Dense)            (None, 1)                 257       
                                                                 
Total params: 194,827
Trainable params: 194,827
Non-trainable params: 0
_________________________________________________________________


In [182]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [183]:
target.count()

144691

In [184]:
train.count()


Area                                      144691
Rainfall                                  144691
State_Name_Andaman and Nicobar Islands    144691
State_Name_Andhra Pradesh                 144691
State_Name_Arunachal Pradesh              144691
                                           ...  
Crop_Wheat                                144691
Crop_Yam                                  144691
Crop_other fibres                         144691
Crop_other misc. pulses                   144691
Crop_other oilseeds                       144691
Length: 153, dtype: int64

In [185]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [186]:
NN_model.fit(train, target, epochs=10, batch_size=256, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/10
449/453 [============================>.] - ETA: 0s - loss: 1046861.2500 - mean_absolute_error: 1046861.2500
Epoch 1: val_loss improved from inf to 45858.72266, saving model to Weights-001--45858.72266.hdf5
453/453 [==============================] - 6s 13ms/step - loss: 1039787.1875 - mean_absolute_error: 1039787.1875 - val_loss: 45858.7227 - val_mean_absolute_error: 45858.7227
Epoch 2/10
452/453 [============================>.] - ETA: 0s - loss: 1040141.2500 - mean_absolute_error: 1040141.2500
Epoch 2: val_loss did not improve from 45858.72266
453/453 [==============================] - 6s 13ms/step - loss: 1039787.1250 - mean_absolute_error: 1039787.1250 - val_loss: 45858.7227 - val_mean_absolute_error: 45858.7227
Epoch 3/10
449/453 [============================>.] - ETA: 0s - loss: 1040906.5000 - mean_absolute_error: 1040906.5000
Epoch 3: val_loss did not improve from 45858.72266
453/453 [==============================] - 6s 13ms/step - loss: 1039787.3125 - mean_absolute_er

In [187]:

from sklearn.preprocessing import MinMaxScaler
import numpy as np
  
# copy the data
df_sklearn = data.copy()
  
# apply normalization techniques
column = 'Area'
tar='Production'
rainfall='Rainfall'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))
df_sklearn[tar] = MinMaxScaler().fit_transform(np.array(df_sklearn[tar]).reshape(-1,1))
df_sklearn[rainfall] = MinMaxScaler().fit_transform(np.array(df_sklearn[rainfall]).reshape(-1,1))
  
# view normalized data  
df_sklearn.corr()

,Crop_Year,Area,Production,Rainfall
Crop_Year,1.000000,-0.025469,0.006935,0.055788
Area,-0.025469,1.000000,0.041603,-0.034916
Production,0.006935,0.041603,1.000000,-0.001535
Rainfall,0.055788,-0.034916,-0.001535,1.000000


In [188]:
# Load wights file of the best model :
wights_file = '/content/Weights-001--45858.72266.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [195]:
predictions = NN_model.predict(test)

In [190]:
print(predictions[100][0])

1.0


In [191]:
data=pd.read_csv("/content/merged - merged.csv")

In [192]:
print(data.Production.isnull().sum())

2258


In [193]:
data.drop(['Unnamed: 0','ANNUAL','Jul-Oct', 'Nov-Mar', 'Apr-Jun', 'Total_Avg'],axis=1,inplace=True)
data.drop(['District_Name'],axis=1,inplace=True)
data.to_csv("train.csv")

In [194]:
print(data)

                         State_Name  Crop_Year      Season  \
0       Andaman and Nicobar Islands       2000      Kharif   
1       Andaman and Nicobar Islands       2000      Kharif   
2       Andaman and Nicobar Islands       2000      Kharif   
3       Andaman and Nicobar Islands       2000  Whole Year   
4       Andaman and Nicobar Islands       2000  Whole Year   
...                             ...        ...         ...   
223055                  West Bengal       2014        Zaid   
223056                  West Bengal       2014        Zaid   
223057                  West Bengal       2014  Whole Year   
223058                  West Bengal       2014        Rabi   
223059                  West Bengal       2014        Rabi   

                       Crop      Area  Production     Rainfall  
0                  Arecanut    1254.0      2000.0   302.100000  
1       Other Kharif pulses       2.0         1.0   302.100000  
2                      Rice     102.0       321.0   302.1000